In [12]:
library(tidyverse)

In [1]:
pigeonpea_df <- read_csv("./rsc/ECO221_Project_2025_Final.csv")
rainfall_df <- read_csv("./rsc/RF_DistrictWise_ECO221_2025.csv")
soil_df <- read_csv("./rsc/Salinity_Alkalinity_ECO221_2025.csv")
names(pigeonpea_df) <- tolower(str_trim(names(pigeonpea_df)))
names(rainfall_df) <- tolower(str_trim(names(rainfall_df)))
names(soil_df) <- tolower(str_trim(names(soil_df)))

pigeonpea_df <- pigeonpea_df %>%
  rename(district = districtname, state = statename)

soil_df <- soil_df %>%
  rename(
    district = `district name`,
    salinity = `average salinity/alkalinity (percentage)`
  )

pigeonpea_df <- pigeonpea_df %>%
  filter(tolower(crop) == "pigeonpea")

pigeonpea_df <- pigeonpea_df %>%
  mutate(irrigatedarea1000hectares = pmin(irrigatedarea1000hectares, area1000hectares, na.rm = TRUE))

# Standardize merge keys
pigeonpea_df <- pigeonpea_df %>%
  mutate(across(c(state, district), ~str_to_lower(str_trim(.))))
rainfall_df <- rainfall_df %>%
  mutate(across(c(state, district), ~str_to_lower(str_trim(.))))
soil_df <- soil_df %>%
  mutate(district = str_to_lower(str_trim(district)))


rainfall_df <- rainfall_df %>%
  select(-year) %>%
  rename(year = yearcode) %>%
  mutate(year = as.integer(year))

pigeonpea_df$year <- as.integer(pigeonpea_df$year)

cat("🔎 Raw values in rainfall_df$month before cleaning:\n")
print(unique(rainfall_df$month))

rainfall_df <- rainfall_df %>%
  mutate(month_clean = str_to_lower(str_trim(str_extract(as.character(month), "^[A-Za-z]+"))))

valid_months <- c("june", "july", "august", "september", "october")
rainfall_df <- rainfall_df %>%
  filter(month_clean %in% valid_months)

print(unique(rainfall_df$month_clean))
rainfall_avg <- rainfall_df %>%
  group_by(state, district, year) %>%
  summarise(rf = mean(rf, na.rm = TRUE), .groups = "drop")

merged_df <- pigeonpea_df %>%
  left_join(rainfall_avg, by = c("state", "district", "year"))

merged_df <- merged_df %>%
  left_join(soil_df %>% select(district, salinity), by = "district")

# Save final merged file
write_csv(merged_df, "Seasonal_Merged_PigeonPea.csv")
cat("\nFinal merge completed. Output saved as: Seasonal_Merged_PigeonPea.csv\n")

print(colSums(is.na(select(merged_df, rf, salinity))))

ERROR: Error in read_csv("./rsc/ECO221_Project_2025_Final.csv"): could not find function "read_csv"
